Config.py

In [ ]:
@dataclass
class RbcParams:
    """RBC model parameters"""
    
    α:     float = 0.36     # Capital share
    η:     float = 0.34     # Labor weight
    ρ:     float = 0.918    # TFP shock persistence
    β:     float = 0.96     # Discount factor
    δ:     float = 0.1      # Depreciation rate
    μ_ϵ:   float = 0.0      # Mean of TFP
    σ_ϵ:   float = 0.014    # Std of TFP 

In [ ]:
import torch
import torch.nn as nn
from dataclass import dataclass, field
from typing import List, Union, Optional


@dataclass
class NetworkParams:
    """ Neural net parameters"""

    in_dim:  int = 2        # input dimension (k_t, a_t)
    hid_dim: List[int] = field(default_factory=lambda: [16]) # Hidden layer
    out_dim: int = 1  # output dimension (consumption share)
    act_fn: Union[str, List[str]] = "relu"  # Activation function(s)
    dropout_rate: float = 0.0 # Dropout rate
    num_batch: int = 100   # Batch size
    num_epoch: int = 5000   # Epoch size
    optimizer: str = "adam"
    out_bound: float = 1e-6  # output bound

    def __post_init__(self):
        """Validate activation functions and optimizers"""

        # Valid activtion functions
        valid_activations = ["relu", "tanh", "sigmoid"]

        if isinstance(self.act_fn, str):
            if self.act_fn.lower() not in valid_activations:
                raise ValueError(f"Activation function must be one of: {valid_activations}")
        else:
            for act in self.act_fn:
                if act.lower() not in valid_activations:
                    raise ValueError(f"Activation function must be one of: {valid_activations}")

        # Valid optimizers
        valid_optimzers = ["adam", "sgd", "swa"]
        if self.optimizer.lower() not in valid_optimizers:
            raise ValueError(f"Optimizer must be one of: {valid_optimizers}")

        
    def get_act_fn(self, idx=0) -> nn.Module:
        """Convert activation string to PyTorch activation fucntion"""
        if isinstance(self.act_fn, list):
            if idx >= len(self.act_fn):
                act_name = self.act_fn[-1]
            else:
                act_name = self.act_fn[idx]
        else:
            act_name = self.act_fn

        act_name = act_name.lower()
        if act_name == "relu":
            return nn.ReLU()
        elif act_name == "tanh":
            return nn.Tanh()
        elif act_name == "sigmoid":
            return nn.Sigmoid()
        else: 
            raise ValueError(f"Unsupported activation: {act_name}")      

Neural network model

In [7]:
class NeuralNetwork(nn.Module):
    """Neural Network model with configurable architecture"""
    def __init__(self, params: NetworkParams):
        """Initialize neural network based on provided parameters"""
        super().__init__()
        self.params = params

        # Initialize layers
        layers = []

        # Input layer
        layers.append(nn.Linear(params.in_dim, params.hid_dim[0]))
        if params.dropout_rate > 0:
            layers.append(nn.Dropout(p=params.dropout_rate))
        layers.append(params.get_act_fn(0))

        # Hidden Layers
        if i in range(1, len(params.hid_dim)):
            layers.append(nn.Linear(params.hid_dim[i-1], params.hid_dim[i]))
            if params.dropout_rate > 0:
                layers.append(nn.Dropout(p=params.dropout_rate))
            layers.append(params.get_act_fn(i))

        # Output Layers
        layers.append(nn.Linear(params.hid_dim[-1], params.out_dim))

        # put the layers together
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        """Forward pass through the network"""
        out = self.layers(x)

        # Apply sigmoid to output
        ζ_0 = torch.sigmoid(out)

        # bound the output
        ζ_1 = torch.minimum(
            torch.maximum(ζ_0, torch.tensor([params.out_bound])),
            torch.tensor([1.0 - params.out_bound])
        )

        return ζ_1

NameError: name 'nn' is not defined